In [1]:
import pandas as pd
import numpy as np

import re
import csv
from io import StringIO
import itertools

In [2]:
class PxFieldDialect(csv.Dialect):
    strict = True
    skipinitialspace = True
    quoting = csv.QUOTE_ALL
    delimiter = ','
    quotechar = '"'
    lineterminator = '\n'

def parse_px_field(field):
    r = csv.reader(StringIO(field), PxFieldDialect())
    
    out = []
    for line in r:
        for item in line:
            # Skip empty strings caused by linebreaks
            if item:
                out.append(item)
        
    return out

In [3]:
filename = "C:/Users/billy/Downloads/20181125051710546145CJQ02.px"
filename2 = "C:/Users/billy/Downloads/2018112521210546594PEA06.px"

In [4]:
with open(filename2) as f:
    data = f.read()

## Separate fields

In [5]:
lines = data.split(";")
fields = [l.strip().split("=", 1) for l in lines if l.strip()]
field_dict = dict(fields)

field_dict

{'CHARSET': '"ANSI"',
 'AXIS-VERSION': '"2006"',
 'LANGUAGE': '"en"',
 'CREATION-DATE': '"20181125 02:01"',
 'DECIMALS': '0',
 'SHOWDECIMALS': '0',
 'MATRIX': '"PEA06"',
 'SUBJECT-AREA': '"Census of Population"',
 'SUBJECT-CODE': '"A01"',
 'TITLE': '"Projected Population 2006 Based (Number) by Criteria for Projection,"\n"Sex, Age and Year"',
 'CONTENTS': '"Projected Population 2006 Based (Number)"',
 'UNITS': '"Number"',
 'STUB': '"Criteria for Projection","Sex","Age"',
 'HEADING': '"Year"',
 'VALUES("Criteria for Projection")': '"Method - M1F1","Method - M1F2","Method - M2F1","Method - M2F2","Method - M0F1",\n"Method - M0F2"',
 'VALUES("Sex")': '"Male","Female"',
 'VALUES("Age")': '"Under 1 year","1 year","2 years","3 years","4 years","5 years","6 years","7 years","8 years","9 years","10 years",\n"11 years","12 years","13 years","14 years","15 years","16 years","17 years","18 years","19 years","20 years","21 years",\n"22 years","23 years","24 years","25 years","26 years","27 years","2

## Parse Headings

In [6]:
head = field_dict["HEADING"].strip("\"")
headers = parse_px_field(field_dict["VALUES(\"{}\")".format(head)])

headers[:5]

['2006', '2007', '2008', '2009', '2010']

## Find & Parse MultiIndex levels

* Find headers

In [7]:
# value_keys = [k for k in field_dict.keys() if k.startswith("VALUES")]
# keys = [re.match(r'VALUES\(\"(.*)\"\)', k).group(1) for k in value_keys]
# levels = [k for k in keys if k != head]

levels = parse_px_field(field_dict["STUB"])
levels

['Criteria for Projection', 'Sex', 'Age']

* Parse values

In [8]:
level_values = []
for lev in levels:
    level_values.append(parse_px_field(field_dict["VALUES(\"{}\")".format(lev)]))

levels_dict = dict(zip(levels, level_values))

levels_dict

{'Criteria for Projection': ['Method - M1F1',
  'Method - M1F2',
  'Method - M2F1',
  'Method - M2F2',
  'Method - M0F1',
  'Method - M0F2'],
 'Sex': ['Male', 'Female'],
 'Age': ['Under 1 year',
  '1 year',
  '2 years',
  '3 years',
  '4 years',
  '5 years',
  '6 years',
  '7 years',
  '8 years',
  '9 years',
  '10 years',
  '11 years',
  '12 years',
  '13 years',
  '14 years',
  '15 years',
  '16 years',
  '17 years',
  '18 years',
  '19 years',
  '20 years',
  '21 years',
  '22 years',
  '23 years',
  '24 years',
  '25 years',
  '26 years',
  '27 years',
  '28 years',
  '29 years',
  '30 years',
  '31 years',
  '32 years',
  '33 years',
  '34 years',
  '35 years',
  '36 years',
  '37 years',
  '38 years',
  '39 years',
  '40 years',
  '41 years',
  '42 years',
  '43 years',
  '44 years',
  '45 years',
  '46 years',
  '47 years',
  '48 years',
  '49 years',
  '50 years',
  '51 years',
  '52 years',
  '53 years',
  '54 years',
  '55 years',
  '56 years',
  '57 years',
  '58 years',
  '

## Parse Data

* Split into stream of cells

In [9]:
data = field_dict["DATA"].replace("\n", "").split()

* Convert into table

In [10]:
lines = [data[i:i+len(headers)] for i in range(0, len(data), len(headers))]

lines

[['31302',
  '33084',
  '34060',
  '35015',
  '35981',
  '36916',
  '37861',
  '38640',
  '39310',
  '39856',
  '40297',
  '40686',
  '40858',
  '40916',
  '40826',
  '40638',
  '40369',
  '39991',
  '39604',
  '39246',
  '38890',
  '38513',
  '38274',
  '38149',
  '38163',
  '38212',
  '38288',
  '38454',
  '38732',
  '39089',
  '39478',
  '39909',
  '40352',
  '40821',
  '41302',
  '41811'],
 ['31113',
  '31652',
  '33433',
  '34410',
  '35365',
  '36331',
  '37217',
  '38162',
  '38941',
  '39611',
  '40158',
  '40550',
  '40940',
  '41112',
  '41170',
  '41081',
  '40844',
  '40576',
  '40198',
  '39812',
  '39454',
  '39099',
  '38722',
  '38483',
  '38358',
  '38372',
  '38422',
  '38498',
  '38664',
  '38942',
  '39299',
  '39688',
  '40119',
  '40562',
  '41031',
  '41513'],
 ['31014',
  '31413',
  '31952',
  '33733',
  '34710',
  '35666',
  '36590',
  '37476',
  '38421',
  '39200',
  '39870',
  '40375',
  '40768',
  '41158',
  '41330',
  '41388',
  '41258',
  '41021',
  '40754

* Create MutliIndex from levels

In [13]:
ind = pd.MultiIndex.from_product(list(levels_dict.values()), names=levels_dict.keys())

* Construct DataFrame

In [15]:
df = pd.DataFrame(lines, columns=headers, index=ind)

* Flatten MultiIndex
* Convert all cell dtypes

In [19]:
df2 = df.reset_index().convert_objects(convert_numeric=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
